# Chat Completion API

## 환경설정
1. openai사이트에서 API KEY 발급
2. google colab - secret탭에 `OPENAI_API_KEY` 등록

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4.1-nano",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반가워요. 어떤 도움이 필요하신가요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥 정보
3. Input Data/Example 입력데이터/예시
4. Output Indicator 출력 지시 사항

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스 AFP 속보 시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4.1-nano",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 힘든 작업으로 끼니를 거르는 노동자들의 애환


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4.1-nano', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길
- 교정 제목: 주말 폭우 예보, 안전을 기원합니다


## 연애 코치 ReAct

In [3]:
def dating_coach(prompt, model='gpt-4.1-nano', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [5]:
print(dating_coach('저 이번주말에 여자친구와 100일인데, 기억에 남을 선물을 하고 싶어요. 뭐가 좋을까요?'))

1. 상황분석:  
이번 주말이 두 분의 100일 기념일이군요. 특별한 날인 만큼 의미 있고 기억에 남을 선물을 고민하는 모습이 정말 감동적입니다. 여자친구분이 어떤 것을 좋아하는지, 어떤 취향인지에 따라 선물 선택이 달라질 수 있지만, 일반적으로 의미와 정성을 담은 선물이 좋겠습니다. 또한, 경험이나 감동을 줄 수 있는 선물도 고려해볼 만합니다.

2. 행동계획:  
- 여자친구의 취향과 관심사를 먼저 파악하세요. 어떤 물건을 좋아하는지, 평소 관심 가졌던 것, 혹은 필요로 하는 것이 있는지 조심스럽게 떠올리세요.  
- 감동을 줄 수 있는 맞춤형 선물을 준비하세요. 예를 들어, 두 분이 함께한 추억이 담긴 포토북이나 영상, 또는 특별한 의미를 담은 편지와 함께하는 선물도 좋습니다.  
- 혹은, 특별한 경험(데이 여행, 스파, 공연 티켓 등)을 선물하는 것도 강력한 감동과 기억에 남을 수 있습니다.  
- 구매 전, 예산을 정하고, 신중하게 선택하세요. 온라인 예약 및 배송일정을 고려해 미리 준비하는 것도 중요합니다.

3. 실행:  
- 여자친구의 스타일과 관심사를 떠올려 적합한 선물을 선택하세요. 예를 들어, 좋아하는 액세서리, 책, 화장품, 또는 취미 관련 용품이 될 수 있습니다.  
- 의미 있는 메시지와 함께, 감성적으로 포장하세요. 손편지 또는 작은 카드에 담긴 마음을 전하는 것도 큰 감동을 줍니다.  
- 함께 할 수 있는 경험선물도 고민하세요. 예를 들어, 둘만의 특별한 데이트 계획과 함께 선물을 주면 더 의미 있어요.  
- 시간을 내어 준비하고, 그날을 기대하며 기대하는 모습도 함께 보여주세요.  

이런 준비와 마음가짐이 여자친구에게 큰 의미와 감동을 줄 것입니다. 100일을 기념하는 특별한 순간을 멋지게 만들어보세요!


In [6]:
print(dating_coach('제가 어제 친구들이 놀다가 여자친구랑 연락을 잘 못해서, 단단히 삐져있는 상황입니다. 이 위기를 어떻게 극복해야 할까요?'))

1. 상황분석:
당신은 친구들과 함께 놀다가 여자친구와 연락이 끊긴 상황에 대해 속상함과 불안함을 느끼고 계신 것 같습니다. 여자친구가 삐진 상태라는 것은 신뢰와 소통이 중요한 연애에서 감정을 상하게 한 일이 있기 때문일 가능성이 높습니다. 이 상황은 서로의 감정을 존중하고 해결책을 찾는 것이 필요합니다. 상대방이 왜 삐쳤는지, 어떤 감정을 가지고 있을지 파악하는 것부터 시작하는 것이 좋겠습니다.

2. 행동계획:
- 여자친구의 감정을 이해하려 노력하고, 사과 또는 설명할 기회를 마련하세요.
- 연락이 끊긴 상태에서 갑작스럽게 해결하려 하기보다 먼저 여자친구가 당신과 대화할 준비가 되도록 시간을 주세요.
- 자연스럽고 솔직한 태도로 그녀에게 진심 어린 메시지 또는 대화를 요청하세요. 예를 들어, "어제 연락이 끊겨서 미안해. 너의 감정을 이해하고 싶어. 언제 편할 때 이야기 나눌 수 있을까?"라고 메시지 보내기.
- 이후, 그녀가 말을 열 때까지 기다리며, 감정을 존중하며 공감하는 태도를 유지하세요.

3. 실행:
- 오늘이나 내일, 편안한 시간에 조심스럽게 그녀에게 연락하세요. 문자, 전화 또는 만남을 제안할 수 있습니다.
- 메시지 예시: "어제 연락이 자꾸 끊겨서 미안해. 네 기분이 상했을까 걱정돼. 네 이야기를 듣고 싶어. 언제 시간 괜찮을까?"
- 대화 시에는 그녀의 감정을 경청하고, 책임감 있게 사과하며, 앞으로의 개선 방안을 함께 논의하세요.
- 그녀가 화를 푸는 데 시간이 필요하다면, 기다리고 이해하는 태도를 보여주세요. 작은 행동이나 말로 그녀를 배려하는 모습을 지속적으로 보여주는 것도 중요합니다.

이 과정을 통해 서로의 감정을 재확인하고 신뢰를 회복하는 데 도움이 되길 바랍니다.


## 냉털마스터 ReAct
- 사용자가 냉장고에 남아있는 음식 재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action 기법을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [13]:
def fridge_raid_master(user_foods, model='gpt-4.1-nano', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 사용자 냉장고의 재료를 가지고 최고의 음식을 만들 수 있는 레시피를 추천하는 챗봇입니다.
현재 상황을 분석하고 실행계획을 세우며 추가 내용이 있을지 확인하는 꼼꼼함을 보여주세요.

## 지시사항

### 상황분석
1. **재료 확인**: 냉장고에 있는 재료 목록을 작성하고, 각 재료의 신선함과 사용 가능 시간을 파악합니다.
2. **요리 컨셉 설정**: 재료를 고려하여 요리의 주제를 정하고 잠재적인 요리 아이디어를 계획합니다.

### 실행계획
3. **목표 변경**: 사용 가능한 재료로 만들고자 하는 요리의 목표를 설정하고, 필요한 정보나 추가 자료를 조사합니다.
4. **전략 개발**: 단계별로 목표를 달성하기 위한 전반적인 요리 계획을 세웁니다.
    단, 사용자가 따라하기 쉽게 단계별 가이드를 작성해야 합니다.

### 검증 및 추가내용 확인
5. **레시피 검토**: 작성된 레시피를 검토하여 정보가 명확하고 완전한지 확인합니다.
6. **수정**: 필요 시 수정하여 최종 레시피를 완성합니다.

## 출력형식

1. 상황분석:
   - 재료 목록 및 상태
   - 요리 컨셉 및 아이디어

2. 실행계획:
   - 목표 및 세부 계획
   - 필요한 추가 자료 및 전략

3. 검증 및 추가내용 확인
   - 레시피의 정확성 검토
   - 수정 및 편집

4. 최종레시피
   - 사용자가 따라하기 쉽게 목록으로 작성
   - 필요한 준비물을 별도로 작성할것!

## Examples

- **Input**:
    사용자의 냉장고에는 현재 [무, 파, 두부]이/가 있습니다.
-   **Output**:
  1. 상황분석: 오늘은 냉장고에 있는 무, 파, 두부를 이용해 한식 스프를 만들어보겠습니다...
  2. 실행계획: 먼저 무와 파를 얇게 썰어 냄비에 넣고 물을 부어...
  3. 검증 및 추가내용 확인: 레시피를 확인한 결과, 무와 두부의 사용 방법에 대한 설명이 추가로 필요합니다...

## Notes

- 요리의 문화적 관련성과 감수성을 고려합니다.
- 레시피는 예상 독자의 요리 지식 수준에 맞게 조정됩니다.

    """

    user_message = f"""
    사용자의 냉장고에는 현재 {user_foods}이/가 있습니다.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [12]:
from IPython.display import Markdown

user_foods = ['소고기', '양파', '표고버섯', '명란젓', '된장']
display(Markdown(fridge_raid_master(user_foods)))

1. 상황분석:
   - 재료 목록 및 상태:
     - 소고기: 신선하며, 냉장고에서 적절한 온도에 저장되어 있어 조리 가능
     - 양파: 적당히 보관되어 있고, 껍질이 깨끗하며 사용 가능
     - 표고버섯: 신선하고 부패 없음, 활용 가능
     - 명란젓: 개봉 후 냉장 보관 중이며, 유통기한 내에 있음
     - 된장: 오랜 저장 가능하며, 아직 사용 가능
   - 요리 컨셉 및 아이디어:
     오늘은 이 다섯 가지 재료를 활용해 한국 전통 맛이 느껴지는 든든한 국 또는 찌개를 만들어보는 것이 좋겠습니다. 돼지고기 대신 소고기를 사용하고, 명란젓과 된장을 활용해 깊은 맛을 내는 찌개 또는 전골을 고려합니다. 표고버섯과 양파를 풍미 있게 채소로 넣고, 소고기와 함께 어우러질 수 있는 요리가 적합하겠습니다.

2. 실행계획:
   - 목표 및 세부 계획:
     목표는 명란젓, 된장을 중심으로 한 소고기 및 버섯 양파 찌개 또는 전골을 만드는 것 입니다. 구체적으로는 된장을 풀어 베이스를 만들고, 명란젓을 넣어 감칠맛을 더하며, 소고기와 표고버섯, 양파를 넣어 끓이면 됩니다.
   - 필요한 추가 자료 및 전략:
     조리 시 명란과 된장의 양 조절, 육수의 진한맛 조합, 그리고 부재료와의 밸런스 등을 고려해야 하며, 맛을 보고 조절하는 과정이 필요합니다.

3. 검증 및 추가내용 확인:
   - 레시피의 정확성 검토:
     현재 제안한 재료를 활용한 찌개 또는 전골 요리의 기본 틀은 적합하며, 세부 조리 방법과 양념 비율이 필요할 수 있습니다.
   - 수정 및 편집:
     명란젓과 된장을 과하게 넣지 않도록 하고, 마지막에 간을 조절하는 과정이 필요함을 추가하겠습니다.

4. 최종레시피
   - 준비물:
     - 소고기: 얇게 썬 것 또는 자른 것 200g
     - 양파: 1개, 채 썰기
     - 표고버섯: 3~4개, 슬라이스
     - 명란젓: 1~2 큰술 (기호에 따라 조절)
     - 된장: 2~3 큰술
     - 물 또는 육수: 600~800ml
     - 선택적: 대파, 다진 마늘, 후추
   - 단계별 요리순서:
     1. 냄비에 물 또는 육수를 붓고 끓이기 시작합니다.
     2. 된장을 넣고 잘 풀어줍니다.
     3. 소고기를 넣고 익히며 살짝 익으면, 양파와 표고버섯을 넣어 끓입니다.
     4. 끓기 시작하면, 명란젓을 넣고 살짝 저어줍니다.
     5. 간을 보고 필요 시 된장이나 명란젓을 조금 더 넣거나 간 후추 등으로 맞춥니다.
     6. 다 익으면 그릇에 담아내고, 대파 또는 다진 마늘을 곁들이면 완성입니다.
   - 필요한 준비물:
     - 냄비
     - 도마와 칼
     - 국자 또는 숟가락
     - 계량 스푼

이제 최종 레시피를 정리했으니, 혹시 더 넣거나 바꾸고 싶은 부분이 있는지 알려주시면 반영하겠습니다!

## 면접 질문 생성 JSON 출력

In [14]:
def job_interview(job_posting, model='gpt-4.1-nano', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [22]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.
"""

    user_message = f"""

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}

<< 채용공고 >>
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [23]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)
print(type(output))
print(output)

<class 'str'>
{
    "hard_skill": [
        {
            "question": "Python을 사용하여 머신러닝 모델을 배포할 때 고려해야 할 주요 요소는 무엇인가요?",
            "answer": "모델 배포 시 고려해야 할 주요 요소로는 모델의 성능, 호환성, 사용자 요구사항, 배포 환경과의 적합성 등이 있습니다. 예를 들어, Python의 다양한 라이브러리를 사용하여 모델을 컨테이너화하고, REST API 또는 gRPC를 통해 서비스할 수 있도록 구성할 수 있습니다. 또한 로깅과 모니터링을 통해 모델의 상태를 실시간으로 확인해야 합니다."
        },
        {
            "question": "Kubernetes와 Docker를 이용한 컨테이너 오케스트레이션 프로세스에 대해 설명해 주세요.",
            "answer": "Docker를 사용하여 애플리케이션과 그 의존성을 컨테이너화한 후, Kubernetes를 사용하여 이러한 컨테이너를 배포하고 관리합니다. Kubernetes는 자동으로 컨테이너를 배포하고, 스케일링하며, 불안정한 컨테이너를 재시작하거나 교체하는 등의 기능을 제공하여 신뢰할 수 있는 애플리케이션 운영을 지원합니다."
        },
        {
            "question": "MLflow를 활용하여 머신러닝 실험 관리 및 모델 서빙을 어떻게 구현하나요?",
            "answer": "MLflow를 사용하여 실험을 기록하고, 다양한 모델을 비교 분석할 수 있습니다. 실험 결과를 MLflow Tracking Server에 저장하고, 모델을 Registry에 보관하여 버전 관리합니다. 이후에는 MLflow Models를 통해 실제 환경에 배포할 수 있으며, REST API를 통해 외부에서 모델에 접근할 수 있도록 설정합니다."
        }
    ],
    "soft_skill": [
        {
 

In [24]:
import json

# 응답데이터가 json형식이 아니라면 오류발생!
data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'question': 'Python을 사용하여 머신러닝 모델을 배포할 때 고려해야 할 주요 요소는 무엇인가요?', 'answer': '모델 배포 시 고려해야 할 주요 요소로는 모델의 성능, 호환성, 사용자 요구사항, 배포 환경과의 적합성 등이 있습니다. 예를 들어, Python의 다양한 라이브러리를 사용하여 모델을 컨테이너화하고, REST API 또는 gRPC를 통해 서비스할 수 있도록 구성할 수 있습니다. 또한 로깅과 모니터링을 통해 모델의 상태를 실시간으로 확인해야 합니다.'}, {'question': 'Kubernetes와 Docker를 이용한 컨테이너 오케스트레이션 프로세스에 대해 설명해 주세요.', 'answer': 'Docker를 사용하여 애플리케이션과 그 의존성을 컨테이너화한 후, Kubernetes를 사용하여 이러한 컨테이너를 배포하고 관리합니다. Kubernetes는 자동으로 컨테이너를 배포하고, 스케일링하며, 불안정한 컨테이너를 재시작하거나 교체하는 등의 기능을 제공하여 신뢰할 수 있는 애플리케이션 운영을 지원합니다.'}, {'question': 'MLflow를 활용하여 머신러닝 실험 관리 및 모델 서빙을 어떻게 구현하나요?', 'answer': 'MLflow를 사용하여 실험을 기록하고, 다양한 모델을 비교 분석할 수 있습니다. 실험 결과를 MLflow Tracking Server에 저장하고, 모델을 Registry에 보관하여 버전 관리합니다. 이후에는 MLflow Models를 통해 실제 환경에 배포할 수 있으며, REST API를 통해 외부에서 모델에 접근할 수 있도록 설정합니다.'}]
[{'question': '팀원과의 협업에서 발생할 수 있는 갈등을 어떻게 해결하나요?', 'answer': '갈등이 발생하면 먼저 문제의 원인을 명확히 파악하고, 관련된 모든 팀원과 대화를 통해 서로의 의견을 존중하며 이해하려고 노력합니다. 갈등을 중재하고 해결하기 위해 열린 소통을 통해 합의를 도출하며, 팀워크 강화를 위해